# Hello Observable World!

https://rxpy.readthedocs.io/en/latest/get_started.html

In [7]:
import reactivex
from reactivex import of, create, operators as ops

In [9]:
def my_first_observable(observer, scheduler):
    observer.on_next("Alpha")
    observer.on_next("Beta")
    observer.on_next("Gamma")
    observer.on_next("Delta")
    observer.on_next("Epsilon")
    observer.on_completed()
    observer.on_error(Exception("ma bad"))

source = create(my_first_observable)

source.subscribe(
    on_next=lambda value: print("Received {0}".format(value)),
    on_error=lambda error: print("Error Occurred: {0}".format(error)),
    on_completed=lambda: print("Done!"),
)

Received Alpha
Received Beta
Received Gamma
Received Delta
Received Epsilon
Done!


## OK now it's time to bring in some market data
### _ccxt_ is a sweet package for doing just that

In [5]:
import ccxt
from reactivex import of, operators as op

exchange = ccxt.kucoin()
exchange.load_markets()

COINS_TO_WATCH = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "XRP/USDT", "LTC/USDT"]

def refresh_tickers():
    source = of(*COINS_TO_WATCH)
    composed = source.pipe(
        op.map(lambda s: exchange.fetch_ticker(s)),
    )
    pairs = {}

    def add_to_pairs(value):
        pairs[value['symbol']] = value
        print(f"Received Value {value}!!")
        
    composed.subscribe(
        on_next=lambda value: add_to_pairs(value),
        on_error=lambda error: print(f"Error Occurred: {error}"),
        on_completed=lambda: print("Done!"),
    )
    return pairs
# .pipe(lambda x: exchange.fetch_ticker(x))

### Sweet! So now we have to work each of those JSON bodies into a prompt for GPT to give us a breakdown of what's going on

In [31]:
import os
from dotenv import load_dotenv
import json
import yaml
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


load_dotenv()

oai_org = "org-VAzYa2yfTTttUMNvCBhluUKn"
oai_key = os.getenv("OPENAI_API_KEY")

gpt = OpenAI(temperature=0.9)
market_prompt = ("give a high-level human summary about the trajectory of the"
    + " trading pair, as well as any more info you need to suggest a "
    + "market strategy:\n::::\n {market_data}")
market_data = pairs["ADA/USDT"]
market_prompt += json.dumps(market_data)
print(gpt(market_prompt))
# prompt = PromptTemplate(
#     input_variables=['product'],
#     template="WHat is a good name for a company that makes {}?"
# )
# foo = prompt.format(product="socks")
# print(foo)
# gpt = OpenAI(openai_api_key=oai_key, openai_organization=oai_org, temperature=0.9)
# text = "What would be a good company name for a company that makes colorful socks?"
# print(gpt(text))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




The trading pair for ADA/USDT has seen some recent volatility, as the high for the pair was 0.3675 and the low was 0.359903. The last price for the pair was 0.364827, which is a 0.43% decrease from the open price of 0.36641. Based on the current market conditions, this could indicate a bearish trajectory for the pair. Market strategy should take into consideration the current volatility and focus on bearish strategies in order to maximize potential profits.


## THis is awesome, now let's try to apply the _PromptTemplate_ Module to organize all these prompts

In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

gpt = OpenAI(temperature=0.75)
prompt_txt = (
    "give a high-level human summary about the trajectory of the" +
    " trading pair, as well as any more info you need to suggest a " +
    "market strategy:\n::::\n {market_data}")

lvl_1_market_prompt = PromptTemplate(
    input_variables=['market_data'],
    template=prompt_txt
)

chain = LLMChain(llm = gpt, prompt=lvl_1_market_prompt)


__And now the secret sauce: composing all the tickers together!__

In [6]:
fresh_data = refresh_tickers()
token_analysis = {}
for k, v in fresh_data.items():
    analysis = chain.run(market_data=v)
    token_analysis[k] = analysis
    print(f"{k}: {analysis}")
# chain.run(market_data=pairs["ADA/USDT"])

Received Value {'symbol': 'BTC/USDT', 'timestamp': 1683690680218, 'datetime': '2023-05-10T03:51:20.218Z', 'high': 27844.0, 'low': 27353.0, 'bid': 27692.3, 'bidVolume': None, 'ask': 27692.4, 'askVolume': None, 'vwap': 27600.36787936475, 'open': 27586.0, 'close': 27692.4, 'last': 27692.4, 'previousClose': None, 'change': 106.4, 'percentage': 0.38, 'average': 27571.00118449, 'baseVolume': 2803.58256862, 'quoteVolume': 77379910.27408636, 'info': {'time': 1683690680218, 'symbol': 'BTC-USDT', 'buy': '27692.3', 'sell': '27692.4', 'changeRate': '0.0038', 'changePrice': '106.4', 'high': '27844', 'low': '27353', 'vol': '2803.58256862', 'volValue': '77379910.27408636', 'last': '27692.4', 'averagePrice': '27571.00118449', 'takerFeeRate': '0.001', 'makerFeeRate': '0.001', 'takerCoefficient': '1', 'makerCoefficient': '1'}}!!
Received Value {'symbol': 'ETH/USDT', 'timestamp': 1683690680132, 'datetime': '2023-05-10T03:51:20.132Z', 'high': 1861.9, 'low': 1831.2, 'bid': 1844.37, 'bidVolume': None, 'ask'